# A brief summary of C++ (11, 14, 17, 20)
<br>
<div style="opacity: 0.8; font-family: Consolas, Monaco, Lucida Console, Liberation Mono, DejaVu Sans Mono, Bitstream Vera Sans Mono, Courier New; font-size: 12px; font-style: italic;">
    ────────
    for more from the author, visit
    <a href="https://github.com/hazemanwer2000">github.com/hazemanwer2000</a>.
    ────────
</div>

## Table of Contents
* [C++11](#C++11)
    * [Uniform Initialization](#uniform-initialization)
    * [The `auto` keyword](#the-auto-keyword)
    * [The `decltype` operator](#the-decltype-operator)
    * [The range-based `for` loop](#the-range-based-for-loop)
    * [The `nullptr` keyword](#the-nullptr-keyword)
    * [Enumeration Classes](#enumeration-classes)
    * [R-value References](#r-value-references)
    * [Lambda Expressions](#lambda-expressions)
* [C++14](#C++14)
* [C++17](#C++17)
* [C++20](#C++20)

<hr>

*Note:* This writing builds upon <a href="https://github.com/hazemanwer2000/gif-quickies/blob/main/brief_summary_of_cpp.ipynb">*A brief summary of C++ (98)*</a>, and discusses feature introductions in modern standards of the C++ language.

## C++11 <a class="anchor" id="c++11"></a>

### Uniform Initialization <a class="anchor" id="uniform-initialization"></a>

Direct initialization using parentheses, `()`, proved sometimes inconsistent. For example:

In [ ]:
int x();           // Interpreted as a function declaration

C++11 introduces a more consistent form of initialization, called *uniform* initialization, using braces, *{}*.

In [ ]:
int x{};           // Default-initialized to '0'

*Note:* Unlike `()`, `{}` does not allow implicit narrowing of values. For example, implicit conversion of `double` to `float` is prohibted.

Additionally, an immutable list, `initializer_list`, is defined in `<initializer_list>`.

If a constructor that accepts an `initializer_list` object exists, then uniform (or, brace) initialization allows for the initialization of the list implicitly by comma-separated values between the opening and closing braces.

This is similar in syntax to how C-style arrays are initialized.

In [26]:
//%cflags: -I.jupyter

#include <iostream>
#include <vector>
#include <initializer_list>
#include "helpers.h"

template <class T>
class Array {
    std::vector<T> v;
public:
    Array(const std::initializer_list<T> &init) : v(init.begin(), init.end()) {}
    Array(int size) : v(size) {}

    void show() { forwardly(v.begin(), v.end()); }
};

int main() {
    Array<int> arr{1, 2, 3};
    arr.show();
}

1, 2, 3,   


In [ ]:
Array<int> arr{1, 2, 3};               /* Equivalent syntax */

Array<int> arr = {1, 2, 3};

Array<int> arr({1, 2, 3});

In [ ]:
Array<int> arr(5);                     /* Array of size '5', default-initialized 'int's to '0' */

Array<int> arr{5};                     /* Array of size '1', consisting of {5} */

*Note:* When an `initializer_list` constructor is missing, `{}` behave similar to `()`.

### The `auto` keyword <a class="anchor" id="the-auto-keyword"></a>

C++11 introduces the `auto` keyword, that may be used as a type, to be implicitly resolved at compile-time.

In [43]:
#include <iostream>

int main() {
    auto x = 5;
    
    std::cout << x;
}

5

In [45]:
#include <iostream>

auto add(int x, int y) {
    return x + y;
}

int main() {
    std::cout << add(1, 2);
}

3

*Note:* Always use copy initialization with the `auto` keyword. In particular, brace initialization resolves `auto` into `initializer_list`.

The `auto` keyword ignores references and top-level constants, but retains low-level constants.

In [ ]:
#include <iostream>

int main() {
    const int y = 6;       /* 'y' is a top-level constant ('const int') */

    auto x = y;            /* auto resolved to 'int' */
    
    x = 7;                 /* Valid */
}      

In [ ]:
#include <iostream>

int main() {
    const int y = 6;       /* '&y' is a low-level constant ('const int *') */

    auto ptr = &y;         /* auto resolved to 'const int *' */
    
    *ptr = 7;              /* Error: Assignment of read-only location */
}      

*Note:* As an example, `const int` and `int * const` are top-level constants, while `const int *` is a low-level constant.

### The `decltype` operator <a class="anchor" id="the-decltype-operator"></a>

C++11 introduces the `decltype` operator, that accepts an expression, and substitutes for its type at compile-time.

In [48]:
#include <iostream>

int main() {
    decltype(5 + 3.2) val = (5 + 3.2);
    
    std::cout << val;
}

8.2

Unlike the `auto` keyword, the `decltype` operator retains all qualifiers, including references and top-level constants.

In [ ]:
int main() {
    const int x = 5;          /* 'x' is a top-level const */
    
    decltype(x) y = 6;        /* Resolves to 'const int' */
    
    y = 7;                    /* Error: Assignment to read-only variable. */
}

### The `constexpr` keyword <a class="anchor" id="the-constexpr-keyword"></a>

C++11 introduces the `constexpr` keyword, placed before a variable declaration gurantees that it is evaluated at compile-time.

When placed before a function, it gurantees that calls to this function are evaluated at compile-time.

In [70]:
#include <iostream>
                                      /* Before Compilation */
constexpr auto add(int x, int y) {
    return x + y;
}

int main() {
    constexpr int res = add(5, 6);
    
    std::cout << res;
}

11

In [66]:
#include <iostream>
                                      /* After Compilation */
int main() {
    std::cout << 11;
}

11

*Note:* One missing feature of `constexpr` functions, is the ability to overload them with non-`constexpr` functions.

A class with a `constexpr` constructor may be used as a literal.

In [64]:
#include <iostream>
                                      /* Before Compilation */
struct Point {
    int x, y;
    
    constexpr Point(int x, int y) : x(x), y(y) {}
};

int main() {
    constexpr Point p(1, 2);
    
    std::cout << "(" << p.x << ", " << p.y << ")";
}

(1, 2)

In [72]:
#include <iostream>
                                      /* After Compilation */
int main() {
    std::cout << "(" << 1 << ", " << 2 << ")";
}

(1, 2)

### The range-based `for` loop <a class="anchor" id="the-range-based-for-loop"></a>

C++11 introduces a range-based `for` loop, that uses `begin` and `end` members of its post-colon argument.

In [79]:
#include <iostream>
#include <vector>

int main() {
    std::vector<int> v {1, 2, 3, 4, 5};

    for (auto &val : v) {
        std::cout << val << ' ';
    }
}

1 2 3 4 5 

*Note:* It is recommended to refrain from using a range-based `for` loop with a built-in array.

### The `nullptr` keyword <a class="anchor" id="the-nullptr-keyword"></a>

C++11 defines the `nullptr` ke|yword, to replace the ambigious zero-defined `NULL` macro.

In [98]:
//%cflags: -w

#include <iostream>

void foo(long int x) {
    std::cout << "foo(long int)\n";
}

void foo(long int *x) {
    std::cout << "foo(long int *)\n";
}

int main() {
    foo(NULL);
    foo(nullptr);
}

foo(long int)
foo(long int *)


*Note:* The statement `delete nullptr;` is harmless.

### Enumeration Classes <a class="anchor" id="enumeration-classes"></a>

C++11 introduces *enumeration classes*, as a scoped alternative to plain enumerations.

In [104]:
#include <iostream>

enum class Grade {High, Average, Low};

void tell_me(Grade grade) {
    switch (grade) {
        case Grade::High:
            std::cout << "He got a really high grade!";
            break;
        case Grade::Average:
            std::cout << "He's kind'a average.";
            break;
        case Grade::Low:
            std::cout << "He needs to pass the corrective exam, first!";
            break;
    }
}

int main() {
    tell_me(Grade::High);
}

He got a really high grade!

You may specify a specific integral type to contain values from the `enum class` type, which is `int` by default.

In [ ]:
enum class Grade : char {High, Average, Low};

*Note:* Implicit conversion from `int` to an `enum class` type, and vice versa, is prohibited.

### R-value References <a class="anchor" id="r-value-references"></a>

C++11 introduces another type of reference, an *rvalue reference*, as opposed to an *lvalue reference*.

In [ ]:
int x = 5;

int &l_ref1 = x;        /* Reference to an l-value */
int &l_ref2 = 6;        /* Error: Cannot assign r-value to l-value reference */

int &&r_ref = x;        /* Error: Cannot assign l-value to r-value reference */
int &&r_ref = 6;        /* Reference to an r-value */

const &cl_ref = x;      /* Constant reference to an l-value */
const &cl_ref = 6;      /* Constant reference to a temporary object, initialized with r-value */

A *move constructor* is a constructor that accepts an rvalue reference to an object of the same class.

It is meant to *steal* the internals of a temporary object, that uses dynamic memory, or other external resources that may be expensive to copy. Since the temporary object will not be used again, it saves us from the overhead of making a *deep-copy* of the temporary object (twice, once on the stack, and another inside the copy constructor), just to destroy it thereafter.

Additionally, an lvalue may be cast as an rvalue reference, using `static_cast`, and when passed to a move constructor, the lvalue is *moved* to another object, and is rendered useless thereafter.

In [248]:
#include <iostream>
#include <iterator>

class Integers {
    int *ptr;
    int size;
public:
    Integers(std::initializer_list<int> ls) {
        size = ls.size();
        ptr = new int(size);
        
        int i = 0;
        for (int item : ls) {
            ptr[i++] = item;
        }
    }
    
    Integers(Integers &&i) {           /* Move constructor */
        ptr = i.ptr;
        size = i.size;
        
        i.ptr = nullptr;
    }
    
    Integers(const Integers &ls) {     /* Copy constructor */
        size = ls.size;
        ptr = new int(size);
        
        int i = 0;
        for (int item : ls) {
            ptr[i++] = item;
        }
    }
    
    int * begin() { return ptr; }
    int * end() { return ptr + size; }
    
    const int * begin() const { return ptr; }
    const int * end() const { return ptr + size; }
};

int main() {
    Integers arr1 {1, 2, 3};
    Integers arr2 {arr1};                              /* Copy constructor (Deep-copy)    */
    Integers arr3 {static_cast<Integers &&>(arr1)};    /* Move constructor (Shallow-copy) */
    
    if (arr1.begin() == nullptr) { std::cout << "'arr1' has been moved.\n"; }
    if (arr2.begin() == nullptr) { std::cout << "'arr2' has been moved.\n"; }
    if (arr3.begin() == nullptr) { std::cout << "'arr3' has been moved.\n"; }
    
    return 0;
}

'arr1' has been moved.


*Note:* When a move constructor is defined, the implicit copy constructor is no longer defined, and similarly for `=`.

*Note:* C++11 defines `move` function in `<utility>`, that returns an rvalue reference to an lvalue.

### Lambda Expressions <a class="anchor" id="lambda-expressions"></a>

A *lambda expression* can be thought of as an anonymous function object.

In [341]:
#include <iostream>
#include <vector>
#include <functional>
                                  /* Implementation using function objects */
                                  /* Pros: Well-defined and re-usable */
template <class T>
class within_range {
    T lower, upper;
public:
    within_range(T lo, T up) : lower(lo), upper(up) {}
    
    bool operator() (T value) {
        return value >= lower && value <= upper;
    }
};

template<class F, class T>
class print_if {
    F fobj;
public:
    print_if(F fobj) : fobj(fobj) {}
    
    void operator() (T value) {
        if (fobj(value)) {
            std::cout << value << ' ';
        }
    }
};

template<class T>
auto m_within_range(T upper, T lower) {
    return within_range<T>(upper, lower);
}

template<class T, template<class> class F>
auto m_print_if(F<T> fobj) {
    return print_if<F<T>, T>(fobj);
}

int main() {
    std::vector<int> v{1, 2, 3, 4, 5, 6, 7, 8, 9};
    
    for_each(v.begin(), v.end(), m_print_if(m_within_range(3, 7)));
}

3 4 5 6 7 

In [344]:
#include <iostream>
#include <vector>
#include <functional>
                                  /* Implementation using lambda expressions */
                                  /* Pros: Short and concise. */

int main() {
    std::vector<int> v{1, 2, 3, 4, 5, 6, 7, 8, 9};
    
    for_each(v.begin(), v.end(), [] (int value) {
        if (value >= 3 && value <= 7) {
            std::cout << value << ' ';
        }
    });
}

3 4 5 6 7 

A lambda expression is composed of three parts:
* Capure list `[]`, which specifies the variables to be captured from the surrounding scope.
* Argument list `()`, which specifies the parameters passed upon call.
* Body `{}`, which contains the statements to be executed with each call.

`[]` is synonymous with the arguments passed to a function object's constructor, but it has its special syntax:

| *Capture List* | *Captures?* |
| :-- | :-- |
| `[]` | None. |
| `[var1, &var2]` | Copies local variable `var1` and references `var2`. |
| `[=, ...]` | Copies of all local variables, except which are specified next, are referenced. 
| `[&, ...]` | References to all local variables, except which are specified next, are copied. |
| `[this]` | References to all member variables. |

*Note:* `this` may be integrated into different capture lists, but the behavior may be inconsistent across compilers and future standards.

By default, the body of a lambda expression may not modify the captured variables, which is synonymous with defining `operator()` to be `const`. To modify the captured variables, `mutable` should preceed the body.

In [353]:
#include <iostream>

class Integer {
    int member;
public:
    Integer(int local) : member(local) {
        ([=, this] () mutable {
            local++;
            member++;
            return local;
        })();
        
        std::cout << "Local: " << local << '\n';
        std::cout << "Member: " << member << '\n';
    }
};

int main() {
    Integer(1);
}

Local: 1
Member: 2


To re-use a function object, allow the compiler to determine its type, by using the `auto` keyword.

In [352]:
#include <iostream>

class Integer {
    int member;
public:
    Integer(int local) : member(local) {
        auto lambda = ([=, this] () mutable {
            local++;
            member++;
            return local;
        });
        
        std::cout << "Lambda-Call: " << lambda() << '\n';
        std::cout << "Lambda-Call: " << lambda() << '\n';
        
        std::cout << "Local: " << local << '\n';
        std::cout << "Member: " << member << '\n';
    }
};

int main() {
    Integer(1);
}

Lambda-Call: 2
Lambda-Call: 3
Local: 1
Member: 3


Alternatively, and to be able to perform recursion, C++11 defines `std::function` in `<functional>`, that may be passed the return type and argument types of a lambda expression, and yield the proper type.

In [362]:
#include <iostream>
#include <functional>

int main() {
    std::function<void(int)> countdown = [&countdown] (int c) {
        std::cout << "Countdown: " << c << '\n';
        if (c > 0) { countdown(--c); }
    };
    
    countdown(3);
}

Countdown: 3
Countdown: 2
Countdown: 1
Countdown: 0


*Note:* The implementation of `std::function` is beyond the scope of this work.

By default, the return type of a lambda expression is determined implicitly. It may, however, be explicitly defined.

In [365]:
#include <iostream>

int main() {
    auto add = [] (double x, double y) -> int { 
        return x + y;
    };
    
    std::cout << add(4.6, 5.6);
}

10

*Note:* If `->` is unemployed, and no parameters are required, `()` may be omitted.